#### Merge Datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

#File path
excel_file_path = 'Excel_files/Task_Data_Scientist_Dataset.xlsx'

# Make excel sheets into dataframes
df_sheet2 = pd.read_excel(excel_file_path, sheet_name='Soc_Dem')
df_sheet3 = pd.read_excel(excel_file_path, sheet_name='Products_ActBalance')
df_sheet4 = pd.read_excel(excel_file_path, sheet_name='Inflow_Outflow')
df_sheet5 = pd.read_excel(excel_file_path, sheet_name='Sales_Revenues')

# Merging the Dataframes based on the column 'Client'
df = pd.merge(df_sheet2, df_sheet3, on='Client', how='outer')
df = pd.merge(df, df_sheet4, on='Client', how='outer')

# Distinguish the full merged dataset and the individual one accounting for 60%
df_merge = pd.merge(df, df_sheet5, on='Client', how='outer')

display(df_merge.shape)
display(df_merge.head())

In [ ]:
duplicate_rows = df_merge.duplicated().sum()

print('Total number of duplicate rows:',duplicate_rows)

In [ ]:
# Create dummy variables for Sex
sex_mapping = {'M': 1, 'F': 0}
df_merge['Sex'] = df_merge['Sex'].replace(sex_mapping)

In [ ]:
# Replace missing values with 0
df_merge = df_merge.replace(np.nan,0)

In [ ]:
# Create Merged excel file

current_directory = os.getcwd()
output_folder_path = 'Excel_files'

# Generating a new excel file with the corresponding excel folder
output_excel_path = os.path.join(output_folder_path, 'Merged_dataset.xlsx')

df_merge.to_excel(output_excel_path, index=False)

#### Exploratory Data Analysis

##### Client Age

In [ ]:
# Review the age distribution

plt.figure(figsize=(4, 4))

sns.histplot(df_merge['Age'], bins=15, edgecolor="b", kde=True)

plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Amount')

plt.show()


There seem to be ages that are quite young for holding accounts at the bank. Hence, I can assume that these values are erroneous.
Here, I will verify how many clients are younger than 10 years old (min. age account opening in Belgium). 
Moreover, I will verify whether there are irregularities of 'Tenure' being larger than the actual age of a client as well.

In [ ]:
# Verify the rows where Age is less than 10 years old

age_threshold = df_merge['Age'] < 10
print("Underage clients:", age_threshold.sum())

In [ ]:
# Verify when Tenure in years is larger than Age

anomaly_tenure = df_merge['Tenure']/12 > df_merge['Age']

print("Number of Tenure anomalies:", anomaly_tenure.sum())

##### Gender

In [ ]:
warnings.filterwarnings("ignore")

df_merge['Sex'] = df_merge['Sex'].astype(str)

plt.figure(figsize=(4, 4))


sns.countplot(x='Sex', data=df_merge, palette={'0.0': 'orange', '1.0': 'black'})

plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Distribution of Male and Female')
plt.show()

##### Sales Revenue

In [ ]:
plt.figure(figsize=(4,4))

columns = np.arange(3)
revenue = [df_merge['Revenue_CL'].sum(), df_merge['Revenue_CC'].sum(), df_merge['Revenue_MF'].sum()]

plt.bar(columns, revenue, align='center', color=["black","orange","red"])
plt.xticks(columns, ['Customer Loan','Credit Card','Mutual Fund'])
plt.ylabel('Revenue (EUR)')
plt.title('Sales Revenue')
plt.show()

In [ ]:
plt.figure(figsize=(4,4))

columns = np.arange(3)
revenue = [df_merge['Sale_CL'].sum(), df_merge['Sale_CC'].sum(), df_merge['Sale_MF'].sum()]

plt.bar(columns, revenue, align='center', color=["black","orange","red"])
plt.xticks(columns, ['Customer Loan','Credit Card','Mutual Fund'])
plt.ylabel('Count')
plt.title('Number of Sales per Product')
plt.show()

#### Training Set per Product

##### Merge Training Set

In [ ]:
# 60% conveying training set, hence excluding df_merge
df_train = pd.merge(df, df_sheet5, on='Client', how='inner')

display(df_train.shape)
display(df_train.head())

##### Gender

In [ ]:
# Replace M/F with dummy variables
df_train['Sex'] = df_train['Sex'].replace(sex_mapping)

In [ ]:
missing_value_sex= df_train['Sex'].isnull().sum()
display(missing_value_sex)

# Replace missing values with 0
df_train = df_train.replace(np.nan,0)

##### Age and Tenure

In [ ]:
# Replacing ages younger than 10 with the mean

mean_age = df_train['Age'].mean()
df_train.loc[df_train['Age'] < 10, 'Age'] = mean_age

mean_tenure = df_train['Tenure'].mean()
df_train.loc[df_train['Tenure']/12 > df_train['Age'],'Tenure'] = int(mean_tenure)

In [ ]:
# Verifying if Age and Tenure are making sense
age_threshold_train = df_train['Age'] < 10
print("Underage clients:", age_threshold_train.sum())

anomaly_tenure_train = df_train['Tenure']/12 > df_train['Age']
print("Number of Tenure anomalies:", anomaly_tenure_train.sum())

##### Types

In [ ]:
df_train[df_train.columns].info()

##### Missing Values

In [ ]:
percent_missing = round(df_train.isnull().sum() * 100 / len(df),2)
missing_value_df = pd.DataFrame({'Missing_Percentage': percent_missing})
missing_value_df.sort_values(by="Missing_Percentage",ascending=False)

##### Training Sets

In [ ]:
# Create full training set for consumer loan
consumer_loan = ['Sale_MF','Sale_CC','Revenue_MF','Revenue_CC','Count_CL','ActBal_CL']
df_train_cl = df_train.drop(consumer_loan,axis=1)

# Create full training set for credit card
credit_card = ['Sale_MF','Sale_CL','Revenue_MF','Revenue_CL','Count_CC','ActBal_CC']
df_train_cc  = df_train.drop(credit_card,axis=1)

# Create full training set for mutual fund
mutual_fund = ['Sale_CC','Sale_CL','Revenue_CC','Revenue_CL','Count_MF','ActBal_MF']
df_train_mf = df_train.drop(mutual_fund,axis=1)

In [ ]:
print(df_train_cl.shape)
print(df_train_cc.shape)
print(df_train_mf.shape)

#### Modeling